<a href="https://colab.research.google.com/github/SeeBook2022/SeeBook-OCR/blob/main/yung_test/Kakao_OCR_test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kakao OCR API 구현 예제 
참조: 
https://developers.kakao.com/docs/latest/ko/vision/dev-guide#ocr

```
requests==2.14.2
Pillow == 5.0.0
```

```
Please run with args: $ python example.py /path/to/image appkey

/content/drive/MyDrive/AIFFEL/OCR/Dataset/book_crop/bookshelf6.jpg 

300a81cbd55e46e42558ac355472adaf
```

In [ ]:
# 라이브러리 버전 확인
# !pip show Pillow

In [ ]:
# cd /content/drive/MyDrive/AIFFEL/OCR/src/ocr_test

In [ ]:
# pwd

'/content'

In [ ]:
%%writefile /content/drive/MyDrive/AIFFEL/OCR/src/ocr_test/kakao_ocr_shell.py
import json

import cv2
import requests
import sys

LIMIT_PX = 1024
LIMIT_BYTE = 1024*1024 # 1MB
LIMIT_BOX = 40

def kakao_ocr_resize(image_path: str):
  """
  ocr detect/recognize api helper
  ocr api 의 제약사항을 넘어서는 이미지는 요청 이전에 전처리가 필요

  pixel 초과  : resize
  용량 초과   : 다른 포멧으로 압축, 이미지 분할 등

  :param image_path: 이미지 파일 경로
  :return:
  """
  image = cv2.imread(image_path)
  height, width, _ = image.shape

  if LIMIT_PX < height or LIMIT_PX < width:
    ratio = float(LIMIT_PX) / max(height, width)
    iamge = cv2.resize(image, None, fx=ratio, fy=ratio)
    height, width, _ = height, width, _ = image.shape

    # api를 사용하기 전에 이미지 resize된 경우, recognize 시 resize된 결과를 사용
    image_path = '{}_reiszed.jpg'.format(image_path)
    cv2.imwrite(image_path, image)

    return image_path
  return None

def kakao_ocr(image_path: str, appkey: str):
  """
  OCR api request example
  :param image_path: 이미지파일 경로
  :param appkey: 카카오 앱 REST API 키
  """
  API_URL = 'https://dapi.kakao.com/v2/vision/text/ocr'

  print('image_path:', type(image_path), image_path)
  print('app KEY:', type(appkey), appkey)

  headers = {'Authorization' : 'KakaoAK {}'.format(appkey)}   # Original source
  # headers = {'Authorization' : 'KakaoAK 300a81cbd55e46e42558ac355472adaf'}   # REST API KEY
  # headers = {'Authorization' : 'KakaoAK 6b4299e42103df2a041062bca5269a02'}   # Admin KEY

  image = cv2.imread(image_path)
  jpeg_image = cv2.imencode('.jpg', image)[1]
  data = jpeg_image.tobytes()

  return requests.post(API_URL, headers=headers, files={'image': data})


def main():
  if len(sys.argv) != 3:
    print("Please run with args: $ python example.py /path/to/image appkey")

  # load image data  
  image_path = "/content/drive/MyDrive/AIFFEL/OCR/src/ocr_test/data/bookshelf.jpeg"

  # kakao API KEY
  appkey = '300a81cbd55e46e42558ac355472adaf'   # REST API KEY
  # image_path, appkey = sys.argv[1], sys.argv[2]   # original source
  
  resize_impath = kakao_ocr_resize(image_path)
  if resize_impath is not None:
    image_path = resize_impath
    print("원본 대신 리사이즈 된 이미지를 사용합니다.")

  # kakao API 에서 bbox 좌표, 인식한 텍스트 출력
  output = kakao_ocr(image_path, appkey).json()
  outputdata = json.dumps(output, ensure_ascii=False, sort_keys=True, indent=2))
  print("[OCR] output:\n{}\n".format(outputdata)

  # 출력 결과를 array 에 저장
  outputdata = json.loads(outputdata)

  for i in range(len(outputdata['results'])):
    x = outputdata['results'][i]['boxes'][0][0]
    y = outputdata['results'][i]['boxes'][0][1]
    w = (outputdata['results'][i]['boxes'][1][0] - x = outputdata['results'][i]['boxes'][0][0])
    h = (outputdata['results'][i]['boxes'][2][1] - x = outputdata['results'][i]['boxes'][0][1])

    # 원본 이미지 출력
    org_image = cv2.imread(image_path)
    # bbox 범위 이미지
    img_trim = org_image[y:y+h, x:x+w]
    cv2_imshow(img_trim)
    print(outputdata['result'][i]['recognition_words'][0])


if __name__ == '__main__':
  main()


Overwriting /content/drive/MyDrive/AIFFEL/OCR/src/ocr_test/kakao_ocr_shell.py


* 작업 디렉토리로 이동

In [ ]:
cd /content/drive/MyDrive/AIFFEL/OCR/src/ocr_test

/content/drive/MyDrive/AIFFEL/OCR/src/ocr_test


In [ ]:
pwd

'/content/drive/MyDrive/AIFFEL/OCR/src/ocr_test'

In [ ]:
import kakao_ocr_shell

SyntaxError: ignored

In [ ]:
%%timeit -r 5
kakao_ocr_shell.main()

In [ ]:
# shell command 
# python kakao_ocr_shell.py /content/drive/MyDrive/AIFFEL/OCR/src/ocr_test/data/image212.png be974432f8b2ee4ca776f96f5bfde04b
# python kakao_ocr_shell.py /content/drive/MyDrive/AIFFEL/OCR/src/ocr_test/data/image212.png 300a81cbd55e46e42558ac355472adaf
# python kakao_ocr_shell.py /content/drive/MyDrive/AIFFEL/OCR/src/ocr_test/data/image212.png e41ae95e70ada29a0530676560a57ba8

In [ ]:
%timeit?

In [ ]:
%%timeit
# OCR 단계 소요 시간 측정
kakao_ocr_shell.image_path = 'image126.jpg'
kakao_ocr_shell.appkey = 'asdfg'

In [ ]:
from google.colab.patches import cv2_imshow
import cv2

img = cv2.imread('/content/drive/MyDrive/AIFFEL/OCR/src/ocr_test/data/image212.png', cv2.IMREAD_UNCHANGED )
cv2_imshow(img)

In [ ]:
a = '123'
type(a)